# Лабораторная работа 5. Алгоритм колонии муравьев для решения Cell Formation Problem (CFP)

## Мельников Евгений 18ПМИ-1, Малышева Екатерина 18ПМИ-2




In [1]:
import time
from google.colab import drive
import os
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
from math import gcd, floor
from random import randint
import time
from itertools import *
from scipy.optimize import linprog
import copy
import random
from tqdm import tqdm

In [2]:
drive.mount('/drive')
os.listdir('/drive/MyDrive/vrp_benchmarks')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


['A-n69-k9.vrp',
 'A-n62-k8.vrp',
 'A-n32-k5.vrp',
 'output',
 'A-n36-k5.vrp',
 'B-n50-k7.vrp',
 'B-n34-k5.vrp',
 'B-n43-k6.vrp',
 'B-n41-k6.vrp',
 'B-n38-k6.vrp',
 'B-n39-k5.vrp',
 'B-n45-k5.vrp',
 'B-n45-k6.vrp',
 'B-n44-k7.vrp',
 'B-n35-k5.vrp',
 'B-n31-k5.vrp',
 'A-n64-k9.vrp',
 'A-n65-k9.vrp',
 'A-n80-k10.vrp',
 'A-n61-k9.vrp',
 'A-n46-k7.vrp',
 'A-n60-k9.vrp',
 'A-n54-k7.vrp',
 'A-n48-k7.vrp',
 'A-n55-k9.vrp',
 'A-n53-k7.vrp',
 'A-n45-k6.vrp',
 'A-n63-k10.vrp',
 'A-n45-k7.vrp',
 'A-n63-k9.vrp',
 'A-n34-k5.vrp',
 'A-n37-k6.vrp',
 'A-n38-k5.vrp',
 'A-n33-k5.vrp',
 'A-n44-k6.vrp',
 'A-n37-k5.vrp',
 'A-n33-k6.vrp',
 'A-n39-k6.vrp',
 'A-n39-k5.vrp',
 'B-n68-k9.vrp',
 'B-n78-k10.vrp',
 'B-n67-k10.vrp',
 'B-n56-k7.vrp',
 'B-n57-k7.vrp',
 'B-n66-k9.vrp',
 'B-n64-k9.vrp',
 'B-n63-k10.vrp',
 'B-n51-k7.vrp',
 'B-n50-k8.vrp',
 'B-n52-k7.vrp',
 'B-n57-k9.vrp']

In [3]:
def get_number_after(string, key_word, delimiter): # returns the integer between key_word and the delimiter
  num_digits = 1
  if string.find(key_word) != -1:
    starting_ind = index = string.find(key_word) + len(key_word)
    index = starting_ind
    while string[index + 1] != delimiter:
      num_digits += 1
      index += 1
    return int(string[starting_ind:starting_ind+num_digits])

  else: 
    print("No such substring")
    return None


class ant_vrp:
  

  def __init__(self, file_path, num_ants=10, max_iter_num=10, alpha=1, beta=1, phi=0.05): # requires a file  
    fin = open(file_path)
    while (1):
      line = fin.readline()       
      if "COMMENT" in line:
        self.num_trucks = get_number_after(line, "No of trucks: ", ',')
        self.optimal_value = get_number_after(line, "Optimal value: ", ')')
      if "DIMENSION" in line:
        self.dimension = get_number_after(line, "DIMENSION : ", '\n')
      if "CAPACITY" in line:
        self.truck_capacity = get_number_after(line, "CAPACITY : ", '\n')
      if "NODE_COORD_SECTION" in line:        
        break

    node_coords = []
    for i in range(self.dimension):
      data = list(map(int, fin.readline().split())) # turns a line into a list of integers 
      node_coords.append((data[1], data[2])) # coordinates of a node
    
    self.adj_mat = np.zeros((self.dimension, self.dimension)) # empty adj mat

    for i in range(self.dimension):
      for j in range(i+1, self.dimension):
        self.adj_mat[i][j] = ((node_coords[i][0] - node_coords[j][0])**2 + (node_coords[i][1] - node_coords[j][1])**2)**(1/2)
        if self.adj_mat[i][j] == 0:
          self.adj_mat[i][j] = 0.001 # if the nodes are at the same place

        self.adj_mat[j][i] = self.adj_mat[i][j]

    fin.readline() # DEMAND_SECTION
    self.demands = []
    for i in range(self.dimension):
      data = list(map(int, fin.readline().split()))
      self.demands.append(data[1])
      if data[1] == 0:  
        self.depot_position = i
    fin.close()

    self.pheromones = np.zeros((self.dimension, self.dimension)) # list of pheromones of each edge
    for i in range(self.dimension):      
      for j in range(i+1, self.dimension):
        self.pheromones[i][j] = 0.2
        self.pheromones[j][i] = 0.2 

    self.num_ants = num_ants # number of ants   
    self.max_iter_num = max_iter_num # number of iterations
    self.alpha = alpha  # pheromone impact
    self.beta = beta # distance impact
    self.phi = phi # survivability of a pheromone
    self.visited_nodes = [] # list of visited nodes
    self.best_cost = 100000000000000 # best cost function overall
    self.best_cost_in_iteration = 10000000000000 # best cost function in an iteration (best ant's result)
    self.best_paths = [] # list of best trucks' paths overall
    self.best_paths_in_iteration = [] # list of best truck's paths in an iteration


  def choose_next_vert(self, current_vert, cargo_left): # returns the next vertex or -1 if there is no vertex that a truck can go to 
    denominator = 0
    candidates = [] # index of the vertex that an ant can go to 
    candidates_prob = [] # probability of visiting a vertex
    # condidates[i] <=> candidates_prob[i]

    for i in range(self.dimension): # calculating the denominator      
      if i != current_vert and i not in self.visited_nodes and (cargo_left - self.demands[i] >= 0) and i != self.depot_position:
        denominator += (self.pheromones[current_vert][i]**self.alpha)*((1/self.adj_mat[current_vert][i])**self.beta)
    
    if denominator > 0:
      for i in range(self.dimension): # calculating probabilities
        if i != current_vert and i not in self.visited_nodes and (cargo_left - self.demands[i] >= 0) and i != self.depot_position:
          probability = ((self.pheromones[current_vert][i]**self.alpha)*((1/self.adj_mat[current_vert][i])**self.beta))/denominator
          candidates.append(i) 
          candidates_prob.append(probability)         

      #print(len(candidates))
      #print(len(candidates_prob))
      return np.random.choice(candidates, p=candidates_prob)  # choosing a vertex based on a probability
    else:
      return -1 


  def step_pheromone_update(self, prev_vert, curr_vert):  # updates after each ant's step 
    #del_pher=0.2
    del_pher = 0.2/self.adj_mat[prev_vert][curr_vert]

    self.pheromones[prev_vert][curr_vert]=(1-self.phi)*self.pheromones[prev_vert][curr_vert] + self.phi * del_pher
    self.pheromones[curr_vert][prev_vert]
  

  def global_pheromone_update(self): # updates after each iteration
    for truck in range(len(self.best_paths_in_iteration)):
      for vert in range(len(self.best_paths_in_iteration[truck])-1):
        del_pher = self.truck_capacity/self.best_cost_in_iteration
        i=self.best_paths_in_iteration[truck][vert]
        j=self.best_paths_in_iteration[truck][vert+1]
        self.pheromones[i][j]=(1-self.phi)*self.pheromones[i][j] + self.phi * del_pher
        self.pheromones[j][i]
  

  def calc_cost(self): # cost function of self.paths
    total_cost = 0
    for truck in range(len(self.paths)):
      for vert in range(len(self.paths[truck])-1):
        total_cost += self.adj_mat[self.paths[truck][vert]][self.paths[truck][vert+1]]
    return total_cost


  def solve(self):
    start_time = time.time()
    for iter_num in range(self.max_iter_num):
      self.best_cost_in_iteration = 10000000000000 # reset
      for ant in range(self.num_ants):
        self.visited_nodes = [] # cleaning up the list of visited edges for each ant
        self.paths = [] # cleaning up the list of paths for each ant
        for truck in range(self.num_trucks): # finding routes for each truck
          current_position = self.depot_position  # the current position of a truck
          cargo_left = self.truck_capacity  # the amount of cargo left in a truck
          self.paths.append([]) # append a list for ant's paths
          self.paths[truck].append(current_position)       
          while (current_position != -1):
            old_position = current_position
            current_position = self.choose_next_vert(current_position, cargo_left)
            if current_position != -1: # if a truck can go to a next depot
              self.paths[truck].append(current_position)
              self.visited_nodes.append(current_position) # adding a node to the list of visited nodes              
              cargo_left -= self.demands[current_position]
              self.step_pheromone_update(old_position, current_position)
            else: # if a truck can't go to a next depot             
              self.paths[truck].append(self.depot_position)
              self.step_pheromone_update(old_position, self.depot_position)

        # checking if ant paths are valid:        
        node_count = 0
        for truck_path in self.paths:
          for node in truck_path:
            node_count += 1
        if node_count - 2*len(self.paths) + 1 == self.dimension:       
          # finding the best path in an iteration
          curr_ant_cost=self.calc_cost()
          if curr_ant_cost  < self.best_cost_in_iteration:
            self.best_paths_in_iteration = self.paths
            self.best_cost_in_iteration = curr_ant_cost
        else:
          print('The solution is not valid')     

      self.global_pheromone_update()

      if self.best_cost_in_iteration < self.best_cost:
        self.best_cost = self.best_cost_in_iteration
        self.best_paths = self.best_paths_in_iteration
    
    return self.optimal_value, self.best_cost, self.best_paths, time.time() - start_time

In [4]:
class benchmarks_vrp:


  def __init__(self, path, num_ants=1, max_iter_num=1, alpha=1, beta=1, phi=0.5, times_repeat=10, verbose=False): # path to a folder, which contains benchmark files
    self.verbose = verbose
    self.path = path
    self.num_ants=num_ants
    self.max_iter_num=max_iter_num
    self.alpha=alpha
    self.beta=beta
    self.phi=phi    
    self.times_repeat = times_repeat


  def check_single_benchmark(self, filename, times_repeat=10): 
    best_cost = 1000000000000
    best_paths = None
    total_time = 0
    for i in range(times_repeat):
      if self.verbose:
        print("Try %s ..." %(i), end=' ')

      algorithm = ant_vrp(self.path + '/' + filename, num_ants=self.num_ants, max_iter_num=self.max_iter_num, alpha=self.alpha, beta=self.beta, phi=self.phi)
      solution, cost, paths, time = algorithm.solve()
      if cost < best_cost:
        best_cost = cost
        best_paths = paths     
      total_time += time

      if self.verbose:
        print("Time: %s, Result: %s" %((time, cost)))
      
    return solution, best_cost, best_paths, total_time/times_repeat


  def report(self):
    time_table = {}
    func_table = {}
    for filename in os.listdir(self.path):
      if filename!='output':
        if self.verbose:
          print("Working on", filename)
          
        file = open(self.path + "/output/" + filename + ".sol", "w")

        solution, best_cost, best_paths, time = self.check_single_benchmark(filename, self.times_repeat)
        if self.verbose:
          print(solution, best_cost, time)

        time_table[filename]=[time]
        func_table[filename]=[solution, best_cost]
        for truck in range(len(best_paths)):
          file.write("Route #%d: "%(truck+1)+" ".join(str(x+1) for x in best_paths[truck]))
          file.write("\n")
        file.write("cost %d"%best_cost)
        file.write("\n")
        file.close()
      
    time_table = pd.DataFrame.from_dict(time_table)
    func_table = pd.DataFrame.from_dict(func_table)
    func_table = func_table.rename(index={0: 'Optimal', 1 : 'Ant algorithm'})
    time_table = time_table.rename(index={0: 'Ant algorithm'})
    return time_table, func_table
      


In [ ]:
bench = benchmarks_vrp("/drive/MyDrive/vrp_benchmarks", num_ants=8, max_iter_num=50, alpha=0.75, beta=5, phi=0.1, times_repeat=10, verbose=True)
time_table, func_table = bench.report()


### Cost function

In [6]:
func_table

,A-n69-k9.vrp,A-n62-k8.vrp,A-n32-k5.vrp,A-n36-k5.vrp,B-n50-k7.vrp,B-n34-k5.vrp,B-n43-k6.vrp,B-n41-k6.vrp,B-n38-k6.vrp,B-n39-k5.vrp,B-n45-k5.vrp,B-n45-k6.vrp,B-n44-k7.vrp,B-n35-k5.vrp,B-n31-k5.vrp,A-n64-k9.vrp,A-n65-k9.vrp,A-n80-k10.vrp,A-n61-k9.vrp,A-n46-k7.vrp,A-n60-k9.vrp,A-n54-k7.vrp,A-n48-k7.vrp,A-n55-k9.vrp,A-n53-k7.vrp,A-n45-k6.vrp,A-n63-k10.vrp,A-n45-k7.vrp,A-n63-k9.vrp,A-n34-k5.vrp,A-n37-k6.vrp,A-n38-k5.vrp,A-n33-k5.vrp,A-n44-k6.vrp,A-n37-k5.vrp,A-n33-k6.vrp,A-n39-k6.vrp,A-n39-k5.vrp,B-n68-k9.vrp,B-n78-k10.vrp,B-n67-k10.vrp,B-n56-k7.vrp,B-n57-k7.vrp,B-n66-k9.vrp,B-n64-k9.vrp,B-n63-k10.vrp,B-n51-k7.vrp,B-n50-k8.vrp,B-n52-k7.vrp,B-n57-k9.vrp
Optimal,1159.000000,1288.000000,784.000000,799.000000,741.000000,788.000000,742.000000,829.000000,805.000000,549.000000,751.000000,678.000000,909.00000,955.000000,672.000000,1401.000000,1174.00000,1763.000000,1034.000000,914.000000,1354.000000,1167.000000,1073.000000,1073.000000,1010.000000,944.000000,1314.000000,1146.000000,1616.000000,778.000000,949.000000,730.000000,661.000000,937.000000,669.000000,742.00000,831.000000,822.000000,1272.000000,1221.0000,1032.000000,707.000000,1153.000000,1316.000000,861.000000,1496.000000,1032.00000,1312.000000,747.000000,1598.000000
Ant algorithm,1538.996832,1561.660277,939.932694,972.925941,900.117579,854.958118,854.779431,887.197161,938.188722,753.089441,881.821853,794.165984,1059.52208,1084.566072,726.251887,1754.999029,1476.05348,2236.951606,1342.045117,1132.153228,1731.236448,1371.230517,1351.759131,1333.124696,1260.206278,1109.157814,1637.074571,1348.651369,1936.939167,901.759881,1126.636991,851.394771,761.306585,1155.572768,848.037028,851.17467,1035.906666,996.698209,1507.374344,1441.0977,1317.631084,937.592161,1362.147703,1531.411099,1061.714914,1784.112003,1102.58279,1483.077598,951.437981,1849.965688


### Time

In [7]:
time_table

,A-n69-k9.vrp,A-n62-k8.vrp,A-n32-k5.vrp,A-n36-k5.vrp,B-n50-k7.vrp,B-n34-k5.vrp,B-n43-k6.vrp,B-n41-k6.vrp,B-n38-k6.vrp,B-n39-k5.vrp,B-n45-k5.vrp,B-n45-k6.vrp,B-n44-k7.vrp,B-n35-k5.vrp,B-n31-k5.vrp,A-n64-k9.vrp,A-n65-k9.vrp,A-n80-k10.vrp,A-n61-k9.vrp,A-n46-k7.vrp,A-n60-k9.vrp,A-n54-k7.vrp,A-n48-k7.vrp,A-n55-k9.vrp,A-n53-k7.vrp,A-n45-k6.vrp,A-n63-k10.vrp,A-n45-k7.vrp,A-n63-k9.vrp,A-n34-k5.vrp,A-n37-k6.vrp,A-n38-k5.vrp,A-n33-k5.vrp,A-n44-k6.vrp,A-n37-k5.vrp,A-n33-k6.vrp,A-n39-k6.vrp,A-n39-k5.vrp,B-n68-k9.vrp,B-n78-k10.vrp,B-n67-k10.vrp,B-n56-k7.vrp,B-n57-k7.vrp,B-n66-k9.vrp,B-n64-k9.vrp,B-n63-k10.vrp,B-n51-k7.vrp,B-n50-k8.vrp,B-n52-k7.vrp,B-n57-k9.vrp
Ant algorithm,23.810175,18.004182,3.655548,4.717776,10.535639,4.117501,7.292749,6.534846,5.37679,5.731041,8.033629,8.273459,7.771565,4.468744,3.496515,19.846901,20.734814,35.628331,17.963008,8.735213,16.898887,13.138314,9.677678,13.592807,12.260057,8.392482,19.050907,8.150838,19.221814,4.226521,5.100011,5.40681,3.948297,7.791475,5.119997,3.976452,5.807447,5.774006,23.277699,32.996442,21.956255,13.802683,14.813324,21.120689,19.671923,18.77781,11.363803,10.438957,11.416238,14.52897
